In [ ]:
import os.path as op
import itertools

import nibabel as nib
from nimare.dataset import Dataset

In [2]:
dsets = ["ALC", "ATS", "CANN", "COC"]
seeds = ["amygdala", "hippocampus", "insula", "striatum", "vmPFC"]
tests = ["1SampletTest", "2SampletTest"]
gsrs = ["on", "off"]
pipes = ["3dttest", "3dlmer", "combat"]

dataset_dict = {}
for dset, seed in itertools.product(dsets, seeds):
    dset_dir = f"/gpfs1/home/m/r/mriedel/pace/dsets/dset-{dset}/derivatives/NIfTIs"
    hemispheres = [""] if seed == "vmPFC" else ["lh", "rh"]
    for hemis in hemispheres:
        hemis_lb = "_hemis-lh" if hemis == "lh" else "_hemis-rh" if hemis == "rh" else ""
        if seed == "amygdala":
            rois = [f"amygdala1{hemis}", f"amygdala2{hemis}", f"amygdala3{hemis}"]
        elif seed == "hippocampus":
            rois = [
                f"hippocampus3solF1{hemis}",
                f"hippocampus3solF2{hemis}",
                f"hippocampus3solF3{hemis}",
            ]
        elif seed == "insula":
            rois = [f"insulaD{hemis}", f"insulaP{hemis}", f"insulaV{hemis}"]
        elif seed == "striatum":
            rois = [
                f"striatumMatchCD{hemis}",
                f"striatumMatchCV{hemis}",
                f"striatumMatchDL{hemis}",
                f"striatumMatchD{hemis}",
                f"striatumMatchR{hemis}",
                f"striatumMatchV{hemis}",
            ]
        elif seed == "vmPFC":
            rois = ["vmPFC1", "vmPFC2", "vmPFC3", "vmPFC4", "vmPFC5", "vmPFC6"]
    
        for roi, test in itertools.product(rois, tests):
            test_lb = "group-average" if test == "1SampletTest" else "control-case"
            for gsr, pipe in itertools.product(gsrs, pipes):
                study = f"dset-{dset}_seed-{seed}{hemis_lb}_roi-{roi}_test-{test}_gsr-{gsr}_pipe-{pipe}"
                img_nm = f"dset-{dset}_seed-{seed}{hemis_lb}_test-{test}_roi-{roi}_pipe-{pipe}_gsr-{gsr}_map-0unthr_img.nii.gz"
                img_path = op.join(dset_dir, img_nm)
                assert op.exists(img_path)

                dataset_dict[study] = {"contrasts": {}}
                dataset_dict[study]["contrasts"][test_lb] = {
                    "images": {
                        "z": img_path,
                    },
                    "metadata": {"sample_sizes": None},
                }

mask = nib.load("/gpfs1/home/m/r/mriedel/pace/templates/tpl-MNI152NLin2009cAsym_res-01_desc-brain_mask.nii.gz")
dataset = Dataset(dataset_dict, mask=mask)
dataset.save("/gpfs1/home/m/r/mriedel/pace/dsets/ibma-pace_dataset.pkl.gz")